# ABM

In [56]:
import time, enum, math
import numpy as np
import pandas as pd
import pylab as plt
from mesa import Agent, Model
from mesa.time import SimultaneousActivation, RandomActivation
from mesa.space import NetworkGrid
from mesa.datacollection import DataCollector
from networkx.algorithms.shortest_paths.generic import has_path
import networkx as nx
import panel as pn
import panel.widgets as pnw
import random
from tqdm import tqdm, trange
from time import sleep

data_path = '../' #set to wherever the data files are, will be used on every input

In [51]:
for i in tqdm(range(100)):
    sleep(0.1)

100%|██████████| 100/100 [00:10<00:00,  9.65it/s]


In [3]:
def kmtoNaut(km):
    return km / 1.852

In [10]:
ports = pd.read_csv((data_path +'ports.csv'))
data = pd.read_csv((data_path + 'distances.csv'))
origin = pd.read_csv((data_path + 'origin_ports.csv'))

In [14]:
distances = data[["prev_port", "next_port", "distance"]]

In [18]:
distances.astype({'prev_port':'int64', 'next_port':'int64'}).dtypes

prev_port      int64
next_port      int64
distance     float64
dtype: object

In [24]:
origin.head()

,Port_Name,Port_Ref,Country,1000TEU (2017),Ref,PROB
0,Shanghai,Shanghai Gang,China,40233,2005.0,0.090629
1,Ningbo-Zhoushan,ZHOUSHAN,China,24607,2232.0,0.055430
2,Hong Kong,HONG KONG,China,20770,3128.0,0.046786
3,Busan,Busan Hang,Korea,20493,2034.0,0.046162
4,Guangzhou,Guangzhou Gang,China,18858,2001.0,0.042479


In [263]:
e = 0
for index, row in origin.iterrows():
    if pd.isna(ports.iloc[index]["INDEX_NO"]):
        e += 1

e

49

In [11]:
N = data["next_port"].tolist()
N = list(set(N))
print(len(N))

2842


In [25]:
print(np.random.choice(origin["Ref"], size=None, replace=True,  p=origin["PROB"]))

2107.0


In [19]:
G = nx.MultiGraph()
G.add_nodes_from(N) #instatiate the ports as nodes of network
for i in range(len(distances)): #create bi-directional edges with an attribute for length 
       G.add_edge(distances.iloc[i][0], distances.iloc[i][1], length=distances.iloc[i][2])
grid = NetworkGrid(G) #Define Mesa Grid as the just created Network to allow for shipping only in routes

In [236]:
random.sample(G.nodes, k=1)[0]

3579

In [59]:
class ShippingNetwork(Model):
    def __init__(self, N, distances, major_ports, S=2):
        N
        self.major_ports = major_ports
        self.num_ships = S
        self.distances = distances
        self.schedule = SimultaneousActivation(self)
        self.running = True

        self.G = nx.MultiGraph()
        self.G.add_nodes_from(N) #instatiate the ports as nodes of network
        
       
        for i in tqdm(range(len(distances)), desc="Building Network" ): #create bi-directional edges with an attribute for length 
            self.G.add_edge(distances.iloc[i][0], distances.iloc[i][1], length=distances.iloc[i][2])
            
        
        self.grid = NetworkGrid(self.G) #Define Mesa Grid as the just created Network to allow for shipping only in routes
       

        #create ability to remove edges mid-model
        def network_change(self, change_type, change_edge):
            if change_type == "add":
                self.G.add_edge(self.change_edge[0], self.change_edge[1], length=self.change_edge[2])
            if change_type == "remove":
                self.G.remove_edge(self.change_edge[0],self.change_edge[1]) #can we identify an edge by node 1, node 2 & distance?
            #update model with new grid    
            return NetworkGrid(self.G), G


        #create agents 
        Ships = []
       
        for i in tqdm(range(self.num_ships), desc="Placing Ships"):
            dest_port = random.sample(self.G.nodes, k=1)[0] # Sample the destination 
            start_port = np.random.choice(self.major_ports["Ref"], size=None, replace=True,  p=self.major_ports["PROB"])
            destin_port = random.sample(self.G.nodes, k=1)[0]
            #We sample the origin port from a list of the 50 biggest ports world, with the prob = TAU of the port / TAU of all origin ports for 2017
            a = Ship(i+1, self, self.G, start_port, destin_port, major_ports)
            self.schedule.add(a)
            #append to list of ships
            Ships.append(a)
        
            #place agent on origin node
            self.grid.place_agent(a, start_port)



        self.datacollector = DataCollector(
            model_reporters={"Graph":"G"},
            agent_reporters={"Position": "position", "Destination":"destination", "Itinerary":"itinerary", "Distance_Traveled":"distance_traveled", "Route":"current_route", "Route Changes":"route_chng" })


    def step(self, change_type='', change_edge=[]):
        #check network for changes
        if change_type != '' :
            self.grid, self.G = network_change(change_type, change_edge)

        self.schedule.step()     #Run each Agents
        self.datacollector.collect(self)

In [53]:
class Ship(Agent):
    def __init__(self, unique_id, model, G, start_port, destin_port, major_ports, s=13.0):
        super().__init__(unique_id, model)
        self.destination = destin_port
        self.state = 0 #0 for active, numbers > 0 for weeks that ships have to "wait" until arrival to port
        self.speed = s*24*1.852 #speed is given in knots, with 1 knot being 1 nautical mile per hour. Since the model works with distances in km, we convert here (1 nm = 1.852m)
        self.position = start_port
        self.init_route = nx.dijkstra_path(G, self.position, self.destination, weight='length') #We keep a copy of the entire itinerary / distance traveled
        self.init_dist = nx.dijkstra_path_length(G,self.position, self.destination, weight='length') 
        self.current_route, self.current_dist = self.init_route, self.init_dist  #For comparison & navigational purposes, we use current route & distance
        self.next_position = self.current_route[1]
        self.itinerary = [self.position]
        self.distance_traveled = 0
        self.unique_id = unique_id
        self.G = G
        self.step_size = self.ident_distance()
        self.route_chng = 0
        self.routes = 0
        self.major_ports = major_ports
    

    def routing(self):
        #implement dijkstra to define shortest route
        route = nx.dijkstra_path(self.G, self.position, self.destination, weight='length')
        travel_distance = nx.dijkstra_path_length(self.G,self.position, self.destination, weight='length')
        #include a way for capacity?
        return route, travel_distance

    def move(self):
        self.next_position = self.current_route[1]
        self.step_size = self.ident_distance() #look up the distance between two cities 
        self.state = self.step_size / self.speed #change state to step amount
        self.current_dist = self.current_dist - self.step_size #adjust current distance minus the distance traveled in the next step
        self.model.grid.move_agent(self, self.next_position) #move the agent
        self.current_route.remove(self.current_route[0]) #remove the next step from the itinerary
        self.position = self.next_position
        if len(self.current_route) == 1:
            self.next_position = self.current_route[0] 
        else:
            self.next_position = self.current_route[1] #update current route

    def ident_distance(self): #look up the distance of the current step
        return self.G.get_edge_data(self.position, self.next_position)[0]['length']
    
    def reroute(self):
        self.routes += 1
        if self.position in self.major_ports["Ref"].values: # We re-route in such a way that ships that have either an origin port or a destination port that is a major port
            self.destination = random.sample(self.G.nodes, k=1)[0]
        else:
            self.destination = np.random.choice(self.major_ports["Ref"], size=None, replace=True,  p=self.major_ports["PROB"])
        
        self.current_route, self.current_dist = self.routing()
        self.state = 3

    

    def step(self):
        self.state = self.state - 1 #'move' ships by one weeks progress
        if self.state <= 0.000: #ships that are en-route to the node they are going to next do not move / perform other activities
            self.distance_traveled += self.step_size #ship has arrived at port, let's add the distance traveled to their 
           
            #add the current position to itinerary
            if self.position != self.destination: #if current stop is not the final stop
                new_route, new_distance = self.routing() #perform a new routing to compare against current routing
                
                if new_route == self.current_route: #if current routing is the same as new, just move (default case)
                    # print("default case")
                    self.move()
                    self.itinerary.append(self.position)
        
            # THIS CURRENTLY ONLY CHANGES THE ROUTE IF THE NEXT STEP IS BLOCKED
                elif new_distance > self.current_dist: #if current route is shorter than newly calculated route, check for obstructions
                    # print('reroute: current route shorter than new route')
                    if not has_path(self.G, self.position, self.next_position): 
                        self.current_route = new_route
                        self.current_dist = new_distance
                        self.route_chng += 1
                    self.move()
                    self.itinerary.append(self.position)
                
                
                else: # final option is that current route is longer than new route (think Suez reopening after a while), here, we just take the new option
                    # print('reroute: current route longer than new route')
                    self.current_route = new_route
                    self.current_dist = new_distance
                    self.route_chng += 1
                    self.move()
                    self.itinerary.append(self.position)
            
            else: #if ship is arrived at final position, get a new route, and start back
                # print('arrival')
                self.reroute()
                self.itinerary.append(self.position)
        # print("Ship: {}, Source: {}, Destination: {}, Position: {},  Next Stop {}, Time until next Stop {}".format(self.unique_id, self.itinerary[0], self.destination, self.position, self.next_position, self.state ))

    
    # def collect_time

    # def collect_costs

In [60]:

model = ShippingNetwork(N, distances, origin, 1)

steps = 60
for i in trange(steps, desc="Stepping Model"):
    model.step()
agent_state = model.datacollector.get_agent_vars_dataframe()


Stepping Model: 100%|██████████| 60/60 [00:05<00:00, 11.88it/s]


In [55]:

agent_state


,,Position,Destination,Itinerary,Distance_Traveled,Route,Route Changes
Step,AgentID,,,,,,
1,1,3365.0,1858.0,"[2088.0, 3365.0, 6869.0, 6868.0, 6972.0, 1858....",31.633634,[1858.0],0
2,1,6869.0,1858.0,"[2088.0, 3365.0, 6869.0, 6868.0, 6972.0, 1858....",63.267268,[1858.0],0
3,1,6868.0,1858.0,"[2088.0, 3365.0, 6869.0, 6868.0, 6972.0, 1858....",65.564482,[1858.0],0
4,1,6972.0,1858.0,"[2088.0, 3365.0, 6869.0, 6868.0, 6972.0, 1858....",66.871488,[1858.0],0
5,1,1858.0,1858.0,"[2088.0, 3365.0, 6869.0, 6868.0, 6972.0, 1858....",69.472331,[1858.0],0
6,1,1858.0,3711.0,"[2088.0, 3365.0, 6869.0, 6868.0, 6972.0, 1858....",263.127115,[3711.0],0
7,1,1858.0,3711.0,"[2088.0, 3365.0, 6869.0, 6868.0, 6972.0, 1858....",263.127115,[3711.0],0
8,1,1858.0,3711.0,"[2088.0, 3365.0, 6869.0, 6868.0, 6972.0, 1858....",263.127115,[3711.0],0
9,1,6972.0,3711.0,"[2088.0, 3365.0, 6869.0, 6868.0, 6972.0, 1858....",456.781899,[3711.0],0


In [127]:
route = nx.dijkstra_path(G,2939, 3688, weight='length')
route

[2939, 2040.0, 6868.0, 6869.0, 7100.0, 27973.0, 2876.0, 3268.0, 3688.0]

In [123]:
 travel_distance = nx.dijkstra_path_length(G,5561, 6165, weight='length')
 travel_distance *

597.5009531952306

In [88]:
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
cmap = ListedColormap(["lightblue", "orange", "green",])

def plot_grid(model,fig,layout='spring',title=''):
    graph = model.G
    if layout == 'kamada-kawai':      
        pos = nx.kamada_kawai_layout(graph)  
    elif layout == 'circular':
        pos = nx.circular_layout(graph)
    else:
        pos = nx.spring_layout(graph, iterations=5, seed=8)  
    plt.clf()
    ax=fig.add_subplot()

    nx.draw(graph, pos, node_size=100, edge_color='gray', with_labels=True,
            alpha=0.9,font_size=14,ax=ax)
    ax.set_title(title)
    return

#example usage
fig,ax=plt.subplots(1,2,figsize=(16,10))
model = ShippingNetwork(N, distances)
model.step(distances)
f=plot_grid(model,fig,layout='kamada-kawai')

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [24]:
model.schedule.

[]